In [1]:
import sys
sys.path.append('..')
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sympy import simplify_logic
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import _tree, export_text

from deep_logic.utils.base import validate_network, set_seed, tree_to_formula
from deep_logic.utils.relunn import get_reduced_model, prune_features
from deep_logic.utils.sigmoidnn import prune_equal_fanin
from deep_logic import logic

results_dir = 'results/xor'
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
concepts = ['c1', 'c2', 'c3', 'c4']
n_rep = 10
tot_epochs = 4001
prune_epochs = 2000

In [2]:
# XOR problem
x_train = torch.tensor([
    [0, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
], dtype=torch.float)
y_train = torch.tensor([0, 1, 1, 0], dtype=torch.long)


x_test = torch.tensor([
    [0, 0.95, 0, 1],
    [0, 0.9, 0.2, 0.7],
    [0.05, 1, 0.5, 1],
    [0.1, 0.8, 0.1, 0.8],
    [0.45, 1, 0.95, 0.9],
    [0, 0.35, 0.95, 0.9],
    [0.95, 0.9, 0.75, 0.2],
    [0.75, 0.2, 0.75, 0.15],
    [0.75, 0.15, 0.75, 0.2],
], dtype=torch.float)
y_test = torch.tensor([1, 1, 1, 1, 1, 0, 0, 1, 1], dtype=torch.long)

In [11]:
def train_nn(x_train, y_train, need_pruning, seed, device, relu=False, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(10, 4),
        torch.nn.ReLU() if relu else torch.nn.LeakyReLU(),
        torch.nn.Linear(4, 2),
        torch.nn.Softmax(dim=1),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train)
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.004 * torch.norm(module.weight, 1)
                loss += 0.004 * torch.norm(module.bias, 1)
                break

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            prune_features(model, n_classes=1, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = torch.argmax(y_pred, dim=1)
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [29]:
def train_psi_nn(x_train, y_train, need_pruning, seed, device, verbose=False):
    set_seed(seed)
    x_train = x_train.to(device)
    y_train = y_train.to(device).to(torch.float)
    layers = [
        torch.nn.Linear(x_train.size(1), 10),
        torch.nn.Sigmoid(),
        torch.nn.Linear(10, 4),
        torch.nn.Sigmoid(),
        torch.nn.Linear(4, 1),
        torch.nn.Sigmoid(),
    ]
    model = torch.nn.Sequential(*layers).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_form = torch.nn.BCELoss()
    model.train()
    for epoch in range(tot_epochs):
        # forward pass
        optimizer.zero_grad()
        y_pred = model(x_train).squeeze()
        # Compute Loss
        loss = loss_form(y_pred, y_train)

        for module in model.children():
            if isinstance(module, torch.nn.Linear):
                loss += 0.00001 * torch.norm(module.weight, 1)

        # backward pass
        loss.backward()
        optimizer.step()

        if epoch > prune_epochs and need_pruning:
            model = prune_equal_fanin(model, 2, validate=True, device=device)
            need_pruning = False
            
        # compute accuracy
        if epoch % 500 == 0 and verbose:
            y_pred_d = y_pred > 0.5
            accuracy = y_pred_d.eq(y_train).sum().item() / y_train.size(0)
            print(f'\t Epoch {epoch}: train accuracy: {accuracy:.4f}')
    
    return model

In [38]:
def c_to_y(method, need_pruning, relu, verbose=False):
    methods = []
    splits = []
    explanations = []
    explanations_inv = []
    model_accuracies = []
    explanation_accuracies = []
    explanation_accuracies_inv = []
    elapsed_times = []
    elapsed_times_inv = []
    for seed in range(n_rep):
        explanation, explanation_inv = '', ''
        explanation_accuracy, explanation_accuracy_inv = 0, 0
        
        print(f'Seed [{seed+1}/{n_rep}]')
        
        if method == 'tree':
            classifier = DecisionTreeClassifier(random_state=seed)
            classifier.fit(x_train.detach().numpy(), y_train.detach().numpy())
            y_preds = classifier.predict(x_test.detach().numpy())
            model_accuracy = accuracy_score(y_test.detach().numpy(), y_preds)

            target_class = 1
            start = time.time()
            explanation = tree_to_formula(classifier, concepts, target_class)
            elapsed_time = time.time() - start

            target_class_inv = 0
            start = time.time()
            explanation_inv = tree_to_formula(classifier, concepts, target_class_inv)
            elapsed_time = time.time() - start
        
        else:
            if method == 'psi':
                # positive class
                target_class = 1
                model = train_psi_nn(x_train, y_train.eq(target_class), need_pruning, seed, device, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds>0.5)
                
            else:
                model = train_nn(x_train, y_train, need_pruning, seed, device, relu, verbose)
                y_preds = model(x_test.to(device)).cpu().detach().numpy()
                model_accuracy = accuracy_score(y_test.cpu().detach().numpy(), y_preds.argmax(axis=1))

            # positive class
            target_class = 1
            start = time.time()
            if method == 'psi':
                global_explanation = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                                   x_train.to(device), y_train.to(device), 
                                                                                   target_class=target_class,
                                                                                   method=method, device=device)
            elapsed_time = time.time() - start
            
            if global_explanation:
                explanation_accuracy, _ = logic.base.test_explanation(global_explanation, target_class, x_test, y_test)
                explanation = logic.base.replace_names(global_explanation, concepts)

            # negative class
            target_class_inv = 0
            if method == 'psi':
                model = train_psi_nn(x_train, y_train.eq(target_class_inv), need_pruning, seed, device, verbose)
            
            start = time.time()
            if method == 'psi':
                global_explanation_inv = logic.generate_fol_explanations(model, device)[0]
            else:
                global_explanation_inv, _, _ = logic.relunn.combine_local_explanations(model, 
                                                                                       x_train.to(device), 
                                                                                       y_train.to(device), 
                                                                                       target_class=target_class_inv,
                                                                                       method=method, device=device)
            elapsed_time_inv = time.time() - start
            if global_explanation_inv:
                explanation_accuracy_inv, _ = logic.base.test_explanation(global_explanation_inv, 
                                                                          target_class_inv, x_test, y_test)
                explanation_inv = logic.base.replace_names(global_explanation_inv, concepts)
        
        if verbose:
            print(f'\t Model\'s accuracy: {model_accuracy:.4f}')
            print(f'\t Class {target_class} - Global explanation: "{explanation}" - Accuracy: {explanation_accuracy:.4f}')
            print(f'\t Elapsed time {elapsed_time}')
            print(f'\t Class {target_class} - Global explanation: "{explanation_inv}" - Accuracy: {explanation_accuracy_inv:.4f}')
            print(f'\t Elapsed time {elapsed_time_inv}')

        methods.append(method)
        splits.append(seed)
        explanations.append(explanation)
        explanations_inv.append(explanation_inv)
        model_accuracies.append(model_accuracy)
        explanation_accuracies.append(explanation_accuracy)
        explanation_accuracies_inv.append(explanation_accuracy_inv)
        elapsed_times.append(elapsed_time)
        elapsed_times_inv.append(elapsed_time_inv)
    
    results = pd.DataFrame({
        'method': methods,
        'split': splits,
        'explanation': explanations,
        'explanation_inv': explanations_inv,
        'model_accuracy': model_accuracies,
        'explanation_accuracy': explanation_accuracies,
        'explanation_accuracy_inv': explanation_accuracies_inv,
        'elapsed_time': elapsed_times,
        'elapsed_time_inv': elapsed_times_inv,
    })
    results.to_csv(os.path.join(results_dir, f'results_{method}.csv'))
    
    return results

# General pruning

In [19]:
method = 'pruning'
need_pruning = True
relu = False
results_pruning = c_to_y(method, need_pruning, relu)
results_pruning

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,pruning,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015618,0.015593
1,pruning,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015622,0.016727
2,pruning,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015628,0.015619
3,pruning,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.000000,0.000000
4,pruning,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015621,0.022135
5,pruning,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.022147,0.015631
6,pruning,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.006506,0.015625
7,pruning,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015622,0.015621
8,pruning,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015622,0.006504
9,pruning,9,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.015622,0.022127


# LIME

In [20]:
method = 'lime'
need_pruning = False
relu = False
results_lime = c_to_y(method, need_pruning, relu)
results_lime

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,lime,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.715419,0.693624
1,lime,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.879597,1.044508
2,lime,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,1.144624,0.809914
3,lime,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.971158,0.932915
4,lime,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.722655,0.735761
5,lime,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.750200,0.732349
6,lime,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.896680,0.977266
7,lime,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.688786,0.668434
8,lime,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.677642,0.670385
9,lime,9,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.0,1.0,0.701071,0.676006


# Relu

In [21]:
method = 'weights'
need_pruning = False
relu = True
results_weights = c_to_y(method, need_pruning, relu)
results_weights

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,weights,0,c1 | c2,(c1 & c2) | (~c1 & ~c2),0.888889,0.888889,1.000000,0.021809,0.015620
1,weights,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.022145,0.015605
2,weights,2,~c1 | ~c2,c1 | ~c2,1.000000,0.888889,0.777778,0.015624,0.015635
3,weights,3,~c1 | ~c2,(c1 & c2) | (~c1 & ~c2),1.000000,0.888889,1.000000,0.015607,0.015623
4,weights,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.015646,0.031221
5,weights,5,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.015620,0.015587
6,weights,6,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.015628,0.015615
7,weights,7,~c1 | ~c2,c2 | ~c1,1.000000,0.888889,0.444444,0.015622,0.015621
8,weights,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.000000,1.000000,1.000000,0.015624,0.015627
9,weights,9,c1 | c2,~c2,0.888889,0.888889,0.666667,0.015622,0.015622


# Psi network

In [39]:
method = 'psi'
need_pruning = True
relu = False
results_psi = c_to_y(method, need_pruning, relu, verbose=True)
results_psi

Seed [1/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.7500
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.5000
	 Epoch 3000: train accuracy: 1.0000
	 Epoch 3500: train accuracy: 1.0000
	 Epoch 4000: train accuracy: 1.0000
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.5000
	 Epoch 1000: train accuracy: 0.5000
	 Epoch 1500: train accuracy: 1.0000
	 Epoch 2000: train accuracy: 1.0000
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Model's accuracy: 1.0000
	 Class 1 - Global explanation: "((c1 & ~c2) | (c2 & ~c1))" - Accuracy: 1.0000
	 Elapsed time 0.0249330997467041
	 Class 1 - Global explanation: "(~c1 & ~c2)" - Accuracy: 0.8889
	 Elapsed time 0.03490614891052246
Seed [2/10]
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.7500
	 Epoch 

	 Epoch 500: train accuracy: 0.7500
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 0.7500
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Epoch 0: train accuracy: 0.5000
	 Epoch 500: train accuracy: 0.5000
	 Epoch 1000: train accuracy: 0.7500
	 Epoch 1500: train accuracy: 0.7500
	 Epoch 2000: train accuracy: 0.7500
	 Epoch 2500: train accuracy: 0.7500
	 Epoch 3000: train accuracy: 0.7500
	 Epoch 3500: train accuracy: 0.7500
	 Epoch 4000: train accuracy: 0.7500
	 Model's accuracy: 0.8889
	 Class 1 - Global explanation: "(c1 | c2)" - Accuracy: 0.8889
	 Elapsed time 0.026927709579467773
	 Class 1 - Global explanation: "(c1 | ~c2)" - Accuracy: 0.7778
	 Elapsed time 0.026927471160888672


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,psi,0,((c1 & ~c2) | (c2 & ~c1)),(~c1 & ~c2),1.000000,1.000000,0.888889,0.024933,0.034906
1,psi,1,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,1.000000,1.000000,0.024933,0.035904
2,psi,2,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,1.000000,1.000000,0.033876,0.034906
3,psi,3,((c1 & ~c2) | (c2 & ~c1)),(c2 | ~c1),1.000000,1.000000,0.444444,0.030917,0.034881
4,psi,4,((c1 & ~c2) | (c2 & ~c1)),((c1 & c2) | (~c1 & ~c2)),1.000000,1.000000,1.000000,0.027924,0.041887
5,psi,5,(c2 & ~c1),((c1 & c2) | (~c1 & ~c2)),0.777778,0.777778,1.000000,0.030911,0.023936
6,psi,6,(c2 & ~c1),(~c1 & ~c2),0.666667,0.777778,0.888889,0.026927,0.026927
7,psi,7,(c1 & ~c2),((c1 & c2) | (~c1 & ~c2)),0.444444,0.444444,1.000000,0.026921,0.027926
8,psi,8,((c1 & ~c2) | (c2 & ~c1)),(c1 & c2),1.000000,1.000000,0.888889,0.030917,0.018950
9,psi,9,(c1 | c2),(c1 | ~c2),0.888889,0.888889,0.777778,0.026928,0.026927


# Decision tree

In [40]:
method = 'decision_tree'
need_pruning = False
relu = False
results_tree = c_to_y(method, need_pruning, relu)
results_tree

Seed [1/10]
Seed [2/10]
Seed [3/10]
Seed [4/10]
Seed [5/10]
Seed [6/10]
Seed [7/10]
Seed [8/10]
Seed [9/10]
Seed [10/10]


,method,split,explanation,explanation_inv,model_accuracy,explanation_accuracy,explanation_accuracy_inv,elapsed_time,elapsed_time_inv
0,decision_tree,0,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.010970,0.010970
1,decision_tree,1,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.010971,0.011968
2,decision_tree,2,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.011969,0.011942
3,decision_tree,3,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.010970,0.011968
4,decision_tree,4,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.010971,0.010971
5,decision_tree,5,c1 | c2,(c1 & c2) | (~c1 & ~c2),1.0,0.888889,1.000000,0.010945,0.011994
6,decision_tree,6,~c1 | ~c2,c2 | ~c1,1.0,0.888889,0.444444,0.011004,0.008973
7,decision_tree,7,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.010971,0.010952
8,decision_tree,8,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.011967,0.011968
9,decision_tree,9,(c1 & ~c2) | (c2 & ~c1),(c1 & c2) | (~c1 & ~c2),1.0,1.000000,1.000000,0.012965,0.010970


# Summary

In [41]:
cols = ['model_accuracy', 'explanation_accuracy', 'explanation_accuracy_inv', 'elapsed_time', 'elapsed_time_inv']
mean_cols = [f'{c}_mean' for c in cols]
sem_cols = [f'{c}_sem' for c in cols]

# pruning
df_mean = results_pruning[cols].mean()
df_sem = results_pruning[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_pruning = pd.concat([df_mean, df_sem])
summary_pruning.name = 'pruning'

# lime
df_mean = results_lime[cols].mean()
df_sem = results_lime[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_lime = pd.concat([df_mean, df_sem])
summary_lime.name = 'lime'

# weights
df_mean = results_weights[cols].mean()
df_sem = results_weights[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_weights = pd.concat([df_mean, df_sem])
summary_weights.name = 'weights'

# psi
df_mean = results_psi[cols].mean()
df_sem = results_psi[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_psi = pd.concat([df_mean, df_sem])
summary_psi.name = 'psi'

# tree
df_mean = results_tree[cols].mean()
df_sem = results_tree[cols].sem()
df_mean.columns = mean_cols
df_sem.columns = sem_cols
summary_tree = pd.concat([df_mean, df_sem])
summary_tree.name = 'tree'

summary = pd.concat([summary_pruning, summary_lime, summary_weights, summary_psi, summary_tree], axis=1).T
summary.columns = mean_cols + sem_cols
summary

,model_accuracy_mean,explanation_accuracy_mean,explanation_accuracy_inv_mean,elapsed_time_mean,elapsed_time_inv_mean,model_accuracy_sem,explanation_accuracy_sem,explanation_accuracy_inv_sem,elapsed_time_sem,elapsed_time_inv_sem
pruning,1.000000,1.000000,1.000000,0.013801,0.014558,0.000000,0.000000,0.000000,0.001934,0.002113
lime,1.000000,1.000000,1.000000,0.814783,0.794116,0.000000,0.000000,0.000000,0.048797,0.044488
weights,0.977778,0.944444,0.888889,0.016895,0.017178,0.014815,0.018519,0.061975,0.000847,0.001560
psi,0.877778,0.888889,0.888889,0.028519,0.030715,0.060745,0.057378,0.054935,0.000941,0.002173
tree,1.000000,0.977778,0.944444,0.011370,0.011268,0.000000,0.014815,0.055556,0.000220,0.000300


In [42]:
summary.to_csv(os.path.join(results_dir, 'summary.csv'))